In [ ]:
##### This code is used for obtaining Pilocarpine model data with rate model for 6 interventions with K_SB suppression #####
if 'RunningFromFigureScript' in locals(): # Checking where from the script was runned. If from
    # figure, the directory path is then different
    %run ./DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ./DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions
else:
    %reset 
    %run ../DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ../DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions 
    
import numpy as np
from tqdm import tqdm

In [ ]:
t_interv_windows=np.array([[-0.1,-0.01],# no intervention
                            [0,365*10], # permanent (until t_end)
                            [0,10*7], # 10 weeks
                            [0,2*7], # 2 weeks
                            [0,5*7], # 5 weeks
                            [2*7,7*7], # 5 weeks with 2 weeks delay
                            [5*7,10*7]]) # 5 weeks with 5 weeks delay 

In [ ]:
for ww in tqdm(range(0,len(t_interv_windows[:,0]))):
    params=load_default_parameters('rate')  # heree we define simulation-specific parameters of simulation duration; external input parameters; number of simulation; output location for data 
    params['t_end']= 365*10          
    params['IBDR_E_duration']= [0,2,2,0]
    params['IBDR_E_amplitude']= [0,1.65,1.,0]
    params['Complex_input']= 'no'
    params['amount_simulations']=  len(t_interv_windows[:,0])
    params['number_simulation']= ww+1
    if 'RunningFromFigureScript' in locals(): # Checking where from the script was runned and creating respective figure path
        params['path']= './Data/Data_F8_Rate_PILOCm_intervKsb/Sim_'
    else:
        params['path']= '../Data/Data_F8_Rate_PILOCm_intervKsb/Sim_'

    locals().update(params)

    t_vec=np.linspace(0,t_end,int(t_end/dt+1))
    I_vec=np.zeros(np.shape(t_vec))
    B_vec=np.zeros(np.shape(t_vec))
    D_vec=np.zeros(np.shape(t_vec))
    R_vec=np.zeros(np.shape(t_vec))

    t_lastseiz = []
    I_vec[0]=IC[0] #  setting initial conditions
    B_vec[0]=IC[1]
    D_vec[0]=IC[2]
    R_vec[0]=IC[3]
    
    K_SB_def = params['K_SB']

    for ii in range(1, len(t_vec)): # solving with forward Euler method  
        
        if t_vec[ii]<t_interv_windows[ww,0] or t_vec[ii]>t_interv_windows[ww,1]:
            params['K_SB'] = K_SB_def
        else: 
            params['K_SB'] = 0.01*K_SB_def # supression of K_SB during the intervention time window
        
        [dIdt, dBdt, dDdt, dRdt] = \
                    dIBDRdt_Rate(t_vec[ii], I_vec[ii-1], B_vec[ii-1], D_vec[ii-1], R_vec[ii-1], params)

        I_vec[ii] = I_vec[ii-1] + dIdt * dt
        B_vec[ii] = B_vec[ii-1] + dBdt * dt
        D_vec[ii] = D_vec[ii-1] + dDdt * dt
        R_vec[ii] = R_vec[ii-1] + dRdt * dt


    results_dict = {'t_vec': t_vec,
                    'I_vec': I_vec,
                    'B_vec': B_vec,
                    'D_vec': D_vec,
                    'R_vec': R_vec}

    filename = path + str(number_simulation)
    np.save(filename, [params, results_dict])
    del ii, filename